# Analyze convergent clustering model performance on train_smaller2, the held out set used to tune the model's hyperparameters

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns
import genetools
from IPython.display import display

In [2]:
from malid import config, logger
import crosseval
from malid.trained_model_wrappers import ConvergentClusterClassifier

# Analyze

In [3]:
for gene_locus in config.gene_loci_used:
    for target_obs_col in config.classification_targets:
        try:
            target_obs_col.confirm_compatibility_with_gene_locus(gene_locus)
            target_obs_col.confirm_compatibility_with_cross_validation_split_strategy(
                config.cross_validation_split_strategy
            )
        except Exception as err:
            # Skip invalid combinations
            logger.warning(f"{err}. Skipping.")
            continue

        models_base_dir = ConvergentClusterClassifier._get_model_base_dir(
            gene_locus=gene_locus, target_obs_column=target_obs_col
        )  # should already exist

        output_base_dir = ConvergentClusterClassifier._get_output_base_dir(
            gene_locus=gene_locus,
            target_obs_column=target_obs_col,
        )  # might not yet exist
        output_base_dir.mkdir(parents=True, exist_ok=True)  # create if needed

        model_output_prefix = models_base_dir / "train_smaller1_model"
        results_output_prefix = output_base_dir / "train_smaller1_model"

        try:
            logger.info(
                f"{gene_locus}, {target_obs_col} from {model_output_prefix} to {results_output_prefix}"
            )

            ## Load and summarize
            experiment_set = crosseval.ExperimentSet.load_from_disk(
                output_prefix=model_output_prefix
            )

            # Remove global fold (we trained global fold model, but now get evaluation scores on cross-validation folds only)
            # TODO: make kdict support: del self.model_outputs[:, fold_id]
            for key in experiment_set.model_outputs[:, -1].keys():
                logger.debug(f"Removing {key} (global fold)")
                del experiment_set.model_outputs[key]

            experiment_set_global_performance = experiment_set.summarize()
            experiment_set_global_performance.export_all_models(
                func_generate_classification_report_fname=lambda model_name: f"{results_output_prefix}.classification_report.{model_name}.txt",
                func_generate_confusion_matrix_fname=lambda model_name: f"{results_output_prefix}.confusion_matrix.{model_name}.png",
                dpi=72,
            )
            combined_stats = (
                experiment_set_global_performance.get_model_comparison_stats()
            )
            combined_stats.to_csv(
                f"{results_output_prefix}.compare_model_scores.tsv",
                sep="\t",
            )
            print(gene_locus, target_obs_col)
            display(combined_stats)

            # Which p values were chosen (varies by locus, model, and fold)? How many disease-associated sequences found?
            for fold_id in config.cross_validation_fold_ids:
                for model_name in [
                    "elasticnet_cv",
                    "rf_multiclass",
                    "linearsvm_ovr",
                ]:
                    clf = ConvergentClusterClassifier(
                        fold_id=fold_id,
                        model_name=model_name,
                        fold_label_train="train_smaller1",
                        gene_locus=gene_locus,
                        target_obs_column=target_obs_col,
                    )
                    p_value = clf.p_value_threshold
                    clusters = clf.cluster_centroids_with_class_specific_p_values
                    feature_names = clf.feature_names_in_
                    print(
                        f"{gene_locus}, fold {fold_id}, {target_obs_col}, {model_name}: best p value = {p_value}. Number of disease associated clusters: {(clusters[feature_names] <= p_value).sum().to_dict()}"
                    )
                print()

        except Exception as err:
            logger.exception(f"{gene_locus}, {target_obs_col} failed with error: {err}")

{"message": "GeneLocus.BCR, TargetObsColumnEnum.disease from /home/maxim/code/immune-repertoire-classification/data/data_v_20231027/in_house_peak_disease_timepoints/convergent_clusters/BCR/disease/train_smaller1_model to /home/maxim/code/immune-repertoire-classification/out/in_house_peak_disease_timepoints/convergent_clusters/BCR/disease/train_smaller1_model", "time": "2023-11-04T03:20:53.378289"}


{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T03:21:05.668270"}


{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T03:21:05.694597"}


{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T03:21:05.718716"}


{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T03:21:05.730904"}


...

{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T03:21:22.513186"}


GeneLocus.TCR TargetObsColumnEnum.disease


,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,Accuracy per fold with abstention,MCC per fold with abstention,Unknown/abstention proportion per fold with abstention,Accuracy global with abstention,MCC global with abstention,Unknown/abstention proportion global with abstention,Abstention label global with abstention,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes
lasso_cv,0.832 +/- 0.022 (in 3 folds),0.834 +/- 0.035 (in 3 folds),0.835 +/- 0.032 (in 3 folds),0.842 +/- 0.042 (in 3 folds),0.553 +/- 0.032 (in 3 folds),0.439 +/- 0.031 (in 3 folds),0.553,0.436,0.551 +/- 0.035 (in 3 folds),0.436 +/- 0.036 (in 3 folds),0.004 +/- 0.007 (in 3 folds),0.551,0.434,0.004,Unknown,246,1,247,0.004049,False
lasso_sklearn_with_lambdamax,0.832 +/- 0.021 (in 3 folds),0.834 +/- 0.034 (in 3 folds),0.835 +/- 0.032 (in 3 folds),0.842 +/- 0.042 (in 3 folds),0.553 +/- 0.020 (in 3 folds),0.438 +/- 0.016 (in 3 folds),0.553,0.435,0.551 +/- 0.024 (in 3 folds),0.435 +/- 0.021 (in 3 folds),0.004 +/- 0.007 (in 3 folds),0.551,0.433,0.004,Unknown,246,1,247,0.004049,False
lasso_cv_lambda1se,0.830 +/- 0.032 (in 3 folds),0.830 +/- 0.044 (in 3 folds),0.831 +/- 0.032 (in 3 folds),0.838 +/- 0.041 (in 3 folds),0.537 +/- 0.031 (in 3 folds),0.427 +/- 0.030 (in 3 folds),0.537,0.422,0.535 +/- 0.034 (in 3 folds),0.425 +/- 0.032 (in 3 folds),0.004 +/- 0.007 (in 3 folds),0.534,0.420,0.004,Unknown,246,1,247,0.004049,False
xgboost,0.827 +/- 0.035 (in 3 folds),0.829 +/- 0.038 (in 3 folds),0.835 +/- 0.033 (in 3 folds),0.842 +/- 0.035 (in 3 folds),0.565 +/- 0.038 (in 3 folds),0.453 +/- 0.033 (in 3 folds),0.565,0.449,0.563 +/- 0.041 (in 3 folds),0.450 +/- 0.037 (in 3 folds),0.004 +/- 0.007 (in 3 folds),0.563,0.447,0.004,Unknown,246,1,247,0.004049,False
elasticnet_cv0.75_lambda1se,0.826 +/- 0.018 (in 3 folds),0.827 +/- 0.028 (in 3 folds),0.825 +/- 0.027 (in 3 folds),0.832 +/- 0.035 (in 3 folds),0.533 +/- 0.013 (in 3 folds),0.424 +/- 0.018 (in 3 folds),0.533,0.422,0.531 +/- 0.016 (in 3 folds),0.421 +/- 0.022 (in 3 folds),0.004 +/- 0.007 (in 3 folds),0.530,0.419,0.004,Unknown,246,1,247,0.004049,False
elasticnet_cv0.75,0.825 +/- 0.020 (in 3 folds),0.826 +/- 0.032 (in 3 folds),0.824 +/- 0.028 (in 3 folds),0.831 +/- 0.038 (in 3 folds),0.537 +/- 0.019 (in 3 folds),0.428 +/- 0.027 (in 3 folds),0.537,0.426,0.535 +/- 0.022 (in 3 folds),0.426 +/- 0.030 (in 3 folds),0.004 +/- 0.007 (in 3 folds),0.534,0.423,0.004,Unknown,246,1,247,0.004049,False
elasticnet0.75_sklearn_with_lambdamax,0.824 +/- 0.020 (in 3 folds),0.826 +/- 0.032 (in 3 folds),0.823 +/- 0.029 (in 3 folds),0.830 +/- 0.039 (in 3 folds),0.541 +/- 0.013 (in 3 folds),0.433 +/- 0.023 (in 3 folds),0.541,0.431,0.539 +/- 0.016 (in 3 folds),0.430 +/- 0.025 (in 3 folds),0.004 +/- 0.007 (in 3 folds),0.538,0.428,0.004,Unknown,246,1,247,0.004049,False
elasticnet_cv_lambda1se,0.824 +/- 0.017 (in 3 folds),0.826 +/- 0.028 (in 3 folds),0.823 +/- 0.020 (in 3 folds),0.830 +/- 0.029 (in 3 folds),0.545 +/- 0.010 (in 3 folds),0.440 +/- 0.020 (in 3 folds),0.545,0.437,0.543 +/- 0.011 (in 3 folds),0.437 +/- 0.021 (in 3 folds),0.004 +/- 0.007 (in 3 folds),0.543,0.434,0.004,Unknown,246,1,247,0.004049,False
elasticnet_cv,0.823 +/- 0.020 (in 3 folds),0.824 +/- 0.032 (in 3 folds),0.823 +/- 0.027 (in 3 folds),0.829 +/- 0.037 (in 3 folds),0.541 +/- 0.010 (in 3 folds),0.433 +/- 0.010 (in 3 folds),0.541,0.431,0.539 +/- 0.013 (in 3 folds),0.430 +/- 0.014 (in 3 folds),0.004 +/- 0.007 (in 3 folds),0.538,0.428,0.004,Unknown,246,1,247,0.004049,False
linearsvm_ovr,0.822 +/- 0.021 (in 3 folds),0.820 +/- 0.033 (in 3 folds),0.825 +/- 0.023 (in 3 folds),0.830 +/- 0.033 (in 3 folds),0.553 +/- 0.024 (in 3 folds),0.436 +/- 0.027 (in 3 folds),0.553,0.435,0.551 +/- 0.025 (in 3 folds),0.434 +/- 0.028 (in 3 folds),0.004 +/- 0.007 (in 3 folds),0.551,0.433,0.004,Unknown,246,1,247,0.004049,False


GeneLocus.TCR, fold 0, TargetObsColumnEnum.disease, elasticnet_cv: best p value = 0.001. Number of disease associated clusters: {'Covid19': 0, 'HIV': 38, 'Healthy/Background': 114, 'Influenza': 29, 'Lupus': 123, 'T1D': 3}
GeneLocus.TCR, fold 0, TargetObsColumnEnum.disease, rf_multiclass: best p value = 0.0005. Number of disease associated clusters: {'Covid19': 0, 'HIV': 13, 'Healthy/Background': 68, 'Influenza': 19, 'Lupus': 74, 'T1D': 1}
GeneLocus.TCR, fold 0, TargetObsColumnEnum.disease, linearsvm_ovr: best p value = 0.001. Number of disease associated clusters: {'Covid19': 0, 'HIV': 38, 'Healthy/Background': 114, 'Influenza': 29, 'Lupus': 123, 'T1D': 3}

GeneLocus.TCR, fold 1, TargetObsColumnEnum.disease, elasticnet_cv: best p value = 0.0005. Number of disease associated clusters: {'Covid19': 1, 'HIV': 5, 'Healthy/Background': 248, 'Influenza': 49, 'Lupus': 98, 'T1D': 1}
GeneLocus.TCR, fold 1, TargetObsColumnEnum.disease, rf_multiclass: best p value = 0.001. Number of disease associ

GeneLocus.TCR, fold 2, TargetObsColumnEnum.disease, rf_multiclass: best p value = 0.0005. Number of disease associated clusters: {'Covid19': 0, 'HIV': 27, 'Healthy/Background': 309, 'Influenza': 21, 'Lupus': 200, 'T1D': 0}
GeneLocus.TCR, fold 2, TargetObsColumnEnum.disease, linearsvm_ovr: best p value = 0.0005. Number of disease associated clusters: {'Covid19': 0, 'HIV': 27, 'Healthy/Background': 309, 'Influenza': 21, 'Lupus': 200, 'T1D': 0}

